# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Часть 10: получение грамматических профилей глаголов
1. Считать подготовленные pickle-файлы с грамматической информацией о токенах
2. Составить представления глаголов на основе их токенов

#### Импорт модулей

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import re
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter

#### Получение списка глаголов

In [3]:
pref_df = pd.read_csv('prefixes_and_lemmas.tsv', sep='\t')
pref_df

,prefix,lemma,freq,freq_0,freq_1,freq_2
0,бе[зс],бездействовать,541,155,255,131
1,бе[зс],бездельничать,308,41,169,98
2,бе[зс],безмолвствовать,726,361,191,174
3,бе[зс],безобразить,149,99,43,7
4,бе[зс],безобразничать,306,110,136,60
...,...,...,...,...,...,...
8440,у,ущемляться,62,8,16,38
8441,у,ущипывать,510,147,222,141
8442,у,уязвлять,1305,606,411,288
8443,у,уяснять,1803,677,740,386


In [4]:
# list of dicts
pref_dict = pref_df.to_dict(orient='records')

In [5]:
verbs_to_search = sorted(set([v['lemma'] for v in pref_dict]))

#### Получение грамматических профилей
Открываем ранее полученные файлы:

In [6]:
with open('presov_verb_dict.pickle', 'rb') as f:
    presov_verb_dict = pickle.load(f)

grams = []
for verb in tqdm(presov_verb_dict):
    for token in presov_verb_dict[verb]:
        grams.append(token['gram'])

grams = sorted(list(set(grams)))
len(grams)

clean_grams = []
for gram in grams:
    no_pos_tag = re.sub('^V[=,]', '', gram)
    no_aspect = re.sub('сов|несов|нп|пе|устар|обсц|разг', '', no_pos_tag)
    no_commas = re.sub(',+', ',', no_aspect).strip('(,=')
    clean_grams.append(no_commas)

len(set(clean_grams))

set(clean_grams)

(Giulianelli et al. 2022):
- Tense : {Past 42, Pres 51}
- VerbForm : {Part 68, Fin 25, Inf 9}
- Mood : {Ind 25}
- Voice : {Pass 17}

Для нашего датасета:
- tense: pres, inpr, past
- verbform: ger, part, inf, fin
- mood: ind, imp
- number: sg, pl
- person: 1p, 2p, 3p
- gender: m, f, n
- voice: act, pass

Другое:
- вид: perf, imperf
- переходность: trans, intrans
- падеж: забить, потому что есть только у причастий
- форма: краткая/полная -- есть только у причастий
- одушевленность: вообще что-то странное

Из словаря частотностей получить вектор и сравнивать как в word2vec

full = {'tense': {'непрош': 0, 'прош': 0},
        'vform': {'прич': 0, 'деепр': 0, 'инф': 0, 'фин': 0},
        'mood': {'пов': 0, 'инд': 0},
        'numb': {'ед': 0, 'мн': 0},
        'pers': {'1-л': 0, '2-л': 0, '3-л': 0},
        'gend': {'муж': 0, 'жен': 0, 'сред': 0},
        'voice': {'действ': 0, 'страд': 0}
        }

In [7]:
presov_gram_dict = {}
gram_list = set(['непрош', 'прош', 'прич', 'деепр', 'инф', 'фин', 'пов', 'изъяв', 'ед', 'мн', '1-л', '2-л', '3-л', 'муж', 'жен', 'сред', 'действ', 'страд'])
# grams_to_ignore = set(['полн', 'кр', 'им', 'вин', 'дат', 'твор', 'пр', 'од', 'неод'])

for verb in tqdm(presov_verb_dict):
    presov_gram_dict[verb] = {'непрош': 0, 'прош': 0,
                              'прич': 0, 'деепр': 0, 'инф': 0, 'фин': 0,
                              'пов': 0, 'изъяв': 0,
                              'ед': 0, 'мн': 0,
                              '1-л': 0, '2-л': 0, '3-л': 0,
                              'муж': 0, 'жен': 0, 'сред': 0,
                              'действ': 0, 'страд': 0}
    for token in presov_verb_dict[verb]:
        gram_info = re.sub('^V[=,]', '', token['gram'])
        # убрать одушевленность
        gram_info = re.sub('сов|несов|нп|пе|устар|обсц|разг', '', gram_info)
        gram_info = re.sub(',+', ',', gram_info).strip('(,=').split(',')
        for gram in gram_info:
            if gram in gram_list:
                presov_gram_dict[verb][gram] += 1
        # не нужно ли это убрать?
        if 'прич' not in gram_info and 'деепр' not in gram_info and 'инф' not in gram_info:
            presov_gram_dict[verb]['фин'] += 1

  0%|          | 0/8445 [00:00<?, ?it/s]

2:80: E501 line too long (156 > 79 characters)
3:80: E501 line too long (87 > 79 characters)
21:80: E501 line too long (91 > 79 characters)


In [8]:
presov_gram_dict['безобразить']

{'непрош': 39,
 'прош': 39,
 'прич': 9,
 'деепр': 4,
 'инф': 19,
 'фин': 67,
 'пов': 2,
 'изъяв': 65,
 'ед': 44,
 'мн': 32,
 '1-л': 1,
 '2-л': 4,
 '3-л': 29,
 'муж': 12,
 'жен': 8,
 'сред': 4,
 'действ': 9,
 'страд': 0}

- вектор просто как последовательность чисел?
- понять, что делать с лакунами в грамматических аспектах

In [9]:
presov_vect_dict = {verb: np.array(list(presov_gram_dict[verb].values()))
                    for verb in presov_gram_dict}

In [10]:
presov_vect_dict

{'бездействовать': array([ 72,  60,  18,   0,  21, 116,   2, 114,  91,  43,   5,   5,  55,
         23,  21,   4,  17,   1]),
 'бездельничать': array([27,  9,  1,  1,  5, 34,  0, 34, 23, 12,  6,  3, 17,  5,  0,  0,  1,
         0]),
 'безмолвствовать': array([110, 197,  34,  24,  43, 260,  11, 249, 219,  75,   3,  15,  59,
        114,  33,  14,  34,   0]),
 'безобразить': array([39, 39,  9,  4, 19, 67,  2, 65, 44, 32,  1,  4, 29, 12,  8,  4,  9,
         0]),
 'безобразничать': array([38, 34,  1,  1, 37, 71,  1, 70, 46, 26,  4,  5, 29, 17,  4,  0,  1,
         0]),
 'безуметь': array([19,  2,  0, 16,  1,  5,  0,  5,  4,  1,  0,  0,  3,  1,  0,  0,  0,
         0]),
 'безумствовать': array([51, 21, 15,  3, 25, 56,  2, 54, 55, 16,  7,  8, 20, 20,  4,  2, 15,
         0]),
 'беспокоить': array([ 791,  735,   63,   32,  796, 1496,   65, 1431, 1158,  401,   98,
         114,  579,  175,  151,  231,   50,   13]),
 'беспокоиться': array([ 562,  458,   20,   42,  775, 2142, 1184,  958, 1039, 

In [11]:
# и дальше все то же, что для word2vec